In [1]:
from symred.unitary_partitioning import AntiCommutingOp

In [2]:
anticommuting = {
'IXX':1,
'IYX':2, 
'IZX':1,
'ZIZ':2,
'ZIY':-2,
}

AC_operator_test = AntiCommutingOp(anticommuting)

print(AC_operator_test)

0.267+0.000j IXX +
0.535+0.000j IYX +
0.267+0.000j IZX +
0.535+0.000j ZIZ +
-0.535+0.000j ZIY


In [3]:
## optional sorting of operator! This can improve circuit reductions in SeqRots approach,
## but not need for contextual subspace stuff

# sort into lexicographical order
AC_operator_test.lexicographical_sort()
print(AC_operator_test)

0.535+0.000j ZIZ +
0.267+0.000j IZX +
-0.535+0.000j ZIY +
0.267+0.000j IXX +
0.535+0.000j IYX


In [4]:
# generate sequence of rotations

## note check_reduction can be expensive... only run as a check if necessary

SeqRots, Pfin = AC_operator_test.gen_seq_rotations(s_index=0, check_reduction=True)

for X_sk, theta_sk in SeqRots:
    print(X_sk, f'angle: {theta_sk:.3f}')
print()
print('term reduced too:', Pfin)

1.000-0.000j ZZY angle: 0.464+0.000j
1.000-0.000j IIX angle: 0.730-0.000j
1.000-0.000j ZXY angle: 0.322+0.000j
1.000-0.000j ZYY angle: 0.564+0.000j

term reduced too: 1.000+0.000j ZIZ


In [5]:
SeqRots

[(1.000-0.000j ZZY, (0.4636476090008061+0j)),
 (1.000-0.000j IIX, (0.7297276562269663-0j)),
 (1.000-0.000j ZXY, (0.32175055439664213+0j)),
 (1.000-0.000j ZYY, (0.5639426413606288+0j))]

In [6]:
# generate LCU

## note check_reduction can be expensive... only run as a check if necessary

LCU_rot, Pfin = AC_operator_test.gen_LCU(s_index=None, check_reduction=True)

print('LCU op:', LCU_rot)
print()
print('term reduced too:', Pfin)

LCU op: 0.796+0.000j III +
-0.000-0.336j IXI +
0.000+0.168j IYI +
-0.000-0.336j ZZZ +
-0.000-0.336j ZZY

term reduced too: 1.000+0.000j IZX


In [7]:
# manual SeqRot reduction with object (needed for use with other methods)
rotated_op = AC_operator_test.copy()
for X_sk, theta_sk in SeqRots:
    rotated_op = rotated_op._rotate_by_single_Pword(X_sk, theta_sk)
    rotated_op= rotated_op.cleanup_zeros()
    
print(rotated_op)


1.000+0.000j ZIZ


In [8]:
# manual LCU reduction with object (needed for use with other methods)

rotated_op = (LCU_rot * AC_operator_test * LCU_rot.conjugate).cleanup_zeros()
print(rotated_op)


1.000+0.000j IZX


# CS-VQE example

In [9]:
from symred.chem import MoleculeBuilder
from symred.projection import QubitTapering, CS_VQE
from symred.symplectic import QuantumState
import json

In [10]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)
#with open('data/score_data.json', 'r') as infile:
#    scoring_data = json.load(infile)
#print(molecule_geometries.keys())

In [11]:
# Set molecule parameters
speciesname = 'Be_STO-3G_SINGLET'
#print(speciesname in scoring_data)
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms  = mol_data['atoms']
coords = mol_data['coords']
basis  = mol_data['basis'] #'6-31g' #'cc-pVDZ' #
spin   = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis)

Molecule geometry:
Be	0	0	0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -14.351880476202023
MP2 energy:  -14.376238850841515
CCSD energy: -14.403650751752787
FCI energy:  -14.403655108067682


Number of qubits: 10


In [12]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated via a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)
print('\nonto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
#sor_tap = taper_hamiltonian.taper_it(aux_operator=sor_ham, ref_state=hf_array)
#sor_tap.coeff_vec/=np.linalg.norm(sor_tap.coeff_vec)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

 1 ZIZIIZIZIZ 
 1 IZIZIZIZIZ 
 1 IIIIZZIIII 
 1 IIIIIIZZII 
 1 IIIIIIIIZZ

and may be rotated via a sequence of Clifford pi/2 rotations

[(1.000+0.000j IIIIYZIIII, None), (1.000+0.000j IIIIIIYZII, None), (1.000+0.000j IIIIIIIIYZ, None), (1.000+0.000j YIZIIZIZIZ, None), (1.000+0.000j IYIZIZIZIZ, None)]

onto the single-qubit Pauli operators

-1 XIIIIIIIII 
-1 IXIIIIIIII 
-1 IIIIXIIIII 
-1 IIIIIIXIII 
-1 IIIIIIIIXI

The symetry sector corresponding with the reference [1 1 1 1 0 0 0 0 0 0] is [1 1 1 1 1]
The tapered Hartree-Fock state is  1.0000000000 |11000>


/home/tweaving/anaconda3/lib/python3.8/site-packages/symred/symplectic/stabilizerop.py:43: ComplexWarning: Casting complex values to real discards the imaginary part
  self.coeff_vec = self.coeff_vec.astype(int)


In [13]:
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap)

In [14]:
print(cs_vqe.clique_operator)

-0.000+0.000j XZIII +
-1.000+0.000j ZIZZZ


In [15]:
cs_vqe.SeqRots

[(1.000-0.000j YZZZZ, (3.14159272509991+0j))]

In [16]:
print(cs_vqe.clique_operator.perform_rotations(cs_vqe.SeqRots))

0.000+0.000j XZIII +
1.000+0.000j ZIZZZ


In [17]:
from symred.symplectic import PauliwordOp
from symred.utils import exact_gs_energy
H_cs = cs_vqe.project_onto_subspace(
    stabilizers=PauliwordOp({'IZZZZ':1}), 
    enforce_clique_operator=True
)
cs_nrg = exact_gs_energy(H_cs.to_sparse_matrix)[0]

print(cs_nrg - molecule.fci_energy)

0.00032650533510292234


In [26]:
op = cs_vqe.symmetry_generators.copy()
print(op)
op+=cs_vqe.C0
op

-1 IZIII 
 1 IIZII 
 1 IIIZI 
 1 IIIIZ


-1 IZIII 
 1 IIZII 
 1 IIIZI 
 1 IIIIZ 
 1 ZIZZZ

# SeqRot

In [22]:
SeqRots, Pfin = AC_op.gen_seq_rotations(s_index=None, check_reduction=True)

for X_sk, theta_sk in SeqRots:
    print(X_sk, f'angle: {theta_sk:.3f}')
print()
print('term reduced too:', Pfin)

1.000-0.000j ZZY angle: -1.107+0.000j
1.000-0.000j ZZZ angle: -0.730+0.000j
1.000-0.000j IYI angle: 0.322+0.000j
1.000-0.000j IXI angle: -0.564+0.000j

term reduced too: 1.000+0.000j IZX


In [23]:
# manual SeqRot reduction with object (needed for use with other methods)
rotated_op = AC_op.copy()
for X_sk, theta_sk in SeqRots:
    rotated_op = rotated_op._rotate_by_single_Pword(X_sk, theta_sk)
    rotated_op= rotated_op.cleanup_zeros()
    
print(rotated_op)

1.000+0.000j IZX


# LCU

In [24]:
LCU_rot, Pfin = AC_op.gen_LCU(s_index=None, check_reduction=True)

print('LCU op:', LCU_rot)
print()
print('term reduced too:', Pfin)

LCU op: 0.796+0.000j III +
-0.000-0.336j IXI +
0.000+0.168j IYI +
-0.000-0.336j ZZZ +
-0.000-0.336j ZZY

term reduced too: 1.000+0.000j IZX


In [25]:
rotated_op = (LCU_rot * AC_op * LCU_rot.conjugate).cleanup_zeros()
print(rotated_op)

1.000+0.000j IZX


# Performance comparison

In [26]:
%%timeit

rotated_op = AC_op.copy()
for X_sk, theta_sk in SeqRots:
    rotated_op = rotated_op._rotate_by_single_Pword(X_sk, theta_sk)
    rotated_op = rotated_op.cleanup_zeros()

1.86 ms ± 38.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%timeit (LCU_rot * ham_tap * LCU_rot.conjugate).cleanup_zeros()

AssertionError: Pauliwords defined for different number of qubits

In [ ]:
print(cs_vqe.symmetry_generators)

In [ ]:
print(cs_vqe.clique_operator)

In [ ]:
print(cs_vqe.noncontextual_operator)

In [ ]:
for row in cs_vqe.r_indices:
    print(row)

In [ ]:
print(cs_vqe.noncontextual_basis)

In [ ]:
op_1 = {'IZX':1, 'YZY':1, 'ZZZ':1, 'XXX':1}

In [ ]:
import numpy as np

A = np.random.randint(0,2, (int(1e4),int(1e4)))
B = np.random.randint(0,2, (int(1e4),int(1e4)))

In [ ]:
%timeit (A-B).nonzero()

In [ ]:
%timeit np.any(A!=B)

In [ ]:
%timeit np.all(A==B)

In [ ]:
%timeit ~np.logical_xor(A, B)

In [ ]:
%timeit np.einsum('ij->', np.logical_xor(A, B).astype(int))==0

In [ ]:
np.einsum('ij->', np.logical_xor(A, B).astype(int))==0

In [ ]:
%timeit np.einsum('ij->', np.logical_xor(A, B))==0

In [ ]:
not np.einsum('ij->', np.logical_xor(A, B))

In [ ]:
np.einsum('ij->', np.logical_xor(A, A).astype(int))==0

In [ ]:
%timeit np.logical_xor(A, B).astype(int).nonzero()

In [ ]:
np.random